In [ ]:
import pandas as pd
import numpy as np

#df = pd.read_csv('~/Downloads/train_concat_95_00001.csv')
#y = pd.read_csv('~/Downloads/y.csv', sep=';', usecols=[1])
df = pd.read_csv('E:\\Downloads\\train_redux2.csv', dtype='int8')
y = pd.read_csv('E:\\Downloads\\y2.csv', sep=';', usecols=[1], dtype='int8')

df.shape

In [2]:
import xgboost as xgb
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import StratifiedKFold
from scipy import interp
from sklearn.feature_selection import RFECV

#param_range = [0.001,0.01,0.1,1,10,100]
param_range = [1]

cv = StratifiedKFold(n_splits=3, random_state=42)

#for i in param_range:
classifier = xgb.XGBClassifier(max_depth=3, n_estimators=200, learning_rate=0.05, random_state=42)
rfecv = RFECV(estimator=classifier, step=250, cv=StratifiedKFold(3), scoring='roc_auc', verbose=1)
output = rfecv.fit(df, y.values.ravel())

print("Optimal number of features : %d" % rfecv.n_features_)

C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

In [6]:
import xgboost as xgb
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import StratifiedKFold
from scipy import interp

#param_range = [0.001,0.01,0.1,1,10,100]
param_range = [1]

cv = StratifiedKFold(n_splits=3, random_state=42)

for i in param_range:
    classifier = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05, random_state=42)

    aucs = []
    tprs = []
    mean_fpr = np.linspace(0, 1, 100)

    for train_index, test_index in cv.split(df, y):
        model = classifier.fit(df.iloc[train_index], y.iloc[train_index].values.ravel())
        if hasattr(model, "predict_proba"):
            probas_ = model.predict_proba(df.iloc[test_index])[:, 1]
        else:  # use decision function
            probas_ = model.decision_function(df.iloc[test_index])
        fpr, tpr, thresholds = roc_curve(y.iloc[test_index], probas_)
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    
    print('param=%s Mean ROC (AUC = %0.3f)' % (str(i), mean_auc))
    print(aucs)

param=1 Mean ROC (AUC = 0.824)
[0.81996171208309665, 0.82509055407997534, 0.82673983425906672]


In [1]:
import xgboost as xgb

classifier = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)
classifier

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)